## Existing code

In [6]:
def table_row_to_text(header, row):
    '''
    use templates to convert table row to text
    '''
    res = ""
    
    if header[0]:
        res += (header[0] + " ")

    for head, cell in zip(header[1:], row[1:]):
        res += ("the " + row[0] + " of " + head + " is " + cell + " ; ")
    
    res = remove_space(res)
    return res.strip()
def remove_space(text_in):
    res = []

    for tmp in text_in.split(" "):
        if tmp != "":
            res.append(tmp)

    return " ".join(res)

In [7]:
import json
f=open("train.json")
train=json.load(f)


In [8]:
train[0]

{'pre_text': ['interest rate to a variable interest rate based on the three-month libor plus 2.05% ( 2.05 % ) ( 2.34% ( 2.34 % ) as of october 31 , 2009 ) .',
  'if libor changes by 100 basis points , our annual interest expense would change by $ 3.8 million .',
  'foreign currency exposure as more fully described in note 2i .',
  'in the notes to consolidated financial statements contained in item 8 of this annual report on form 10-k , we regularly hedge our non-u.s .',
  'dollar-based exposures by entering into forward foreign currency exchange contracts .',
  'the terms of these contracts are for periods matching the duration of the underlying exposure and generally range from one month to twelve months .',
  'currently , our largest foreign currency exposure is the euro , primarily because our european operations have the highest proportion of our local currency denominated expenses .',
  'relative to foreign currency exposures existing at october 31 , 2009 and november 1 , 2008 , 

In [10]:
for each_data in train:
    f = open("contextfile.txt", "a")
    pre_text = each_data["pre_text"]
    post_text = each_data["post_text"]
    all_text = pre_text + post_text
    table = each_data["table"]

    gold_indss=each_data["qa"]
    context = all_text+table+list(gold_indss.items())
    print(context)
    f.writelines(str(context))
    f.close()

    break

['interest rate to a variable interest rate based on the three-month libor plus 2.05% ( 2.05 % ) ( 2.34% ( 2.34 % ) as of october 31 , 2009 ) .', 'if libor changes by 100 basis points , our annual interest expense would change by $ 3.8 million .', 'foreign currency exposure as more fully described in note 2i .', 'in the notes to consolidated financial statements contained in item 8 of this annual report on form 10-k , we regularly hedge our non-u.s .', 'dollar-based exposures by entering into forward foreign currency exchange contracts .', 'the terms of these contracts are for periods matching the duration of the underlying exposure and generally range from one month to twelve months .', 'currently , our largest foreign currency exposure is the euro , primarily because our european operations have the highest proportion of our local currency denominated expenses .', 'relative to foreign currency exposures existing at october 31 , 2009 and november 1 , 2008 , a 10% ( 10 % ) unfavorable 

In [11]:
max_len=290
stride=30
for each_data in train:
    f = open("contextfile.txt", "a")
    pre_text = each_data["pre_text"]
    post_text = each_data["post_text"]
    table = each_data["table"]
    gold_inds = each_data["qa"]["gold_inds"]
    
    gold_indss=each_data["qa"]
    table_text = ""
    for row in table[1:]:
        this_row_text = table_row_to_text(table[0], row)
        table_text += (this_row_text + " ")
    table_text = table_text.strip()
    pos_windows = []
    neg_windows = []

    all_text = " ".join(pre_text) + " " + table_text + " " + " ".join(post_text) +"".join(str(list(gold_indss.items())))
    all_text = all_text.replace(". . . . . .", "")
    all_text = all_text.replace(". . . ", "")
    all_text = all_text.replace("* * * * * *", "")

   
    print(all_text)
    break

interest rate to a variable interest rate based on the three-month libor plus 2.05% ( 2.05 % ) ( 2.34% ( 2.34 % ) as of october 31 , 2009 ) . if libor changes by 100 basis points , our annual interest expense would change by $ 3.8 million . foreign currency exposure as more fully described in note 2i . in the notes to consolidated financial statements contained in item 8 of this annual report on form 10-k , we regularly hedge our non-u.s . dollar-based exposures by entering into forward foreign currency exchange contracts . the terms of these contracts are for periods matching the duration of the underlying exposure and generally range from one month to twelve months . currently , our largest foreign currency exposure is the euro , primarily because our european operations have the highest proportion of our local currency denominated expenses . relative to foreign currency exposures existing at october 31 , 2009 and november 1 , 2008 , a 10% ( 10 % ) unfavorable movement in foreign cur

In [12]:
import openai
import os


#openai.api_key  ="sk-"

In [13]:
from llama_index import SimpleDirectoryReader, GPTListIndex, readers, GPTVectorStoreIndex, LLMPredictor, PromptHelper, ServiceContext,LangChainLLM
from langchain import OpenAI
import sys
import os
from IPython.display import Markdown, display

def construct_index(directory_path):
    # set maximum input size
    max_input_size = 4096
    # set number of output tokens
    num_outputs = 2000
    # set maximum chunk overlap
    max_chunk_overlap = 20
    # set chunk size limit
    chunk_size_limit = 600

    # define prompt helper
    prompt_helper = PromptHelper(max_input_size, num_outputs, chunk_overlap_ratio =0.1, chunk_size_limit=chunk_size_limit)

    # define LLM
    #llm_predictor = LLMPredictor(llm=OpenAI(temperature=0.5, model_name="text-davinci-003", max_tokens=num_outputs))
    llm = LangChainLLM(  HuggingFaceTextGenInference(inference_server_url="http://localhost:1234"),  query_wrapper_prompt=query_wrapper_prompt)

    documents = SimpleDirectoryReader(directory_path).load_data()

    service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor, prompt_helper=prompt_helper)
    index = GPTVectorStoreIndex.from_documents(documents, service_context=service_context)

    index.save_to_disk('index.json')

    return index

def ask_chatbot():
    index = GPTSimpleVectorIndex.load_from_disk('index.json')
    while True:
        query = input("What do you want to ask? ")
        response = index.query(query)
        display(Markdown(f"Response: <b>{response.response}</b>"))


ImportError: cannot import name 'LangChainLLM' from 'llama_index' (C:\ProgramData\Anaconda3\envs\myenv\Lib\site-packages\llama_index\__init__.py)

In [ ]:
construct_index('./context')

In [ ]:
ask_chatbot()

In [3]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load pre-trained GPT-2 model and tokenizer
model_name = "gpt2"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Function to generate response based on user input
def generate_response(context, question):
    # Combine context and question with a separator token
    input_text = context + tokenizer.eos_token + question + tokenizer.eos_token
    input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=1024, truncation=True)
    
    # Generate response
    output = model.generate(input_ids, max_length=1024, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    
    return response

# Load context from a file
def load_context(file_path):
    with open(file_path, 'r') as file:
        context = file.read()
    return context

# Define file path for context
context_folder = "context/"
context_file = "contextfile.txt"
context_path = context_folder + context_file

# Load context
context = load_context(context_path)

# Example question
question = "what is the the interest expense in 2009?"

# Generate response
response = generate_response(context, question)
print("Bot:", response)

C:\ProgramData\Anaconda3\envs\myenv\Lib\site-packages\transformers\generation\utils.py:1141: UserWarning: Input length of input_ids is 1024, but `max_length` is set to 1024. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


Bot: interest rate to a variable interest rate based on the three-month libor plus 2.05% ( 2.05 % ) ( 2.34% ( 2.34 % ) as of october 31, 2009 ).if libor changes by 100 basis points, our annual interest expense would change by $ 3.8 million.foreign currency exposure as more fully described in note 2i.in the notes to consolidated financial statements contained in item 8 of this annual report on form 10-k, we regularly hedge our non-u.s.dollar-based exposures by entering into forward foreign currency exchange contracts.the terms of these contracts are for periods matching the duration of the underlying exposure and generally range from one month to twelve months.currently, our largest foreign currency exposure is the euro, primarily because our european operations have the highest proportion of our local currency denominated expenses.relative to foreign currency exposures existing at october 31, 2009 and november 1, 2008, a 10% ( 10 % ) unfavorable movement in foreign currency exchange ra

In [5]:
import torch
from transformers import BertForQuestionAnswering, BertTokenizer

# Load pre-trained BERT model and tokenizer
model_name = "bert-large-uncased-whole-word-masking-finetuned-squad"  # Using a large pre-trained model fine-tuned on SQuAD
model = BertForQuestionAnswering.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained(model_name)

# Function to generate response based on user input
def generate_response(context, question):
    inputs = tokenizer.encode_plus(question, context, return_tensors='pt', max_length=512, truncation=True)
    input_ids = inputs["input_ids"].to(model.device)
    token_type_ids = inputs["token_type_ids"].to(model.device)

    start_scores, end_scores = model(input_ids, token_type_ids=token_type_ids)

    all_tokens = tokenizer.convert_ids_to_tokens(input_ids.squeeze())
    answer_tokens = all_tokens[torch.argmax(start_scores) : torch.argmax(end_scores)+1]
    answer = tokenizer.convert_tokens_to_string(answer_tokens)

    return answer

# Load context from a file
def load_context(file_path):
    with open(file_path, 'r') as file:
        context = file.read()
    return context

# Define file path for context
context_folder = "context/"
context_file = "contextfile.txt"
context_path = context_folder + context_file

# Load context
context = load_context(context_path)

# Example question
question = "what is the the interest expense in 2009?"

# Generate response
response = generate_response(context, question)
print("Bot:", response)


config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

C:\ProgramData\Anaconda3\envs\myenv\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\hp\.cache\huggingface\hub\models--bert-large-uncased-whole-word-masking-finetuned-squad. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


TypeError: argmax(): argument 'input' (position 1) must be Tensor, not str

In [14]:
from openai import OpenAI

In [2]:
client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key="sk-",
)
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0 )
    return response.choices[0].message["content"]


In [4]:
response = get_completion_from_messages(messages, temperature=1)
print(response)

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [36]:
def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context) 
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})))
 
    return pn.Column(*panels)

In [40]:
import panel as pn  # GUI
pn.extension()

panels = [] # collect display 

context = [ {'role':'system', 'content':context  # accumulate messages


inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [16]:
f=open("train.json")
data=json.load(f)
data[0]

{'pre_text': ['interest rate to a variable interest rate based on the three-month libor plus 2.05% ( 2.05 % ) ( 2.34% ( 2.34 % ) as of october 31 , 2009 ) .',
  'if libor changes by 100 basis points , our annual interest expense would change by $ 3.8 million .',
  'foreign currency exposure as more fully described in note 2i .',
  'in the notes to consolidated financial statements contained in item 8 of this annual report on form 10-k , we regularly hedge our non-u.s .',
  'dollar-based exposures by entering into forward foreign currency exchange contracts .',
  'the terms of these contracts are for periods matching the duration of the underlying exposure and generally range from one month to twelve months .',
  'currently , our largest foreign currency exposure is the euro , primarily because our european operations have the highest proportion of our local currency denominated expenses .',
  'relative to foreign currency exposures existing at october 31 , 2009 and november 1 , 2008 , 

In [ ]:

        acc_len = 0
        all_text_in = {}
        all_table_in = {}

        for tmp in gold_inds:
            if "table" in tmp:
                all_table_in[tmp] = gold_inds[tmp]
            else:
                all_text_in[tmp] = gold_inds[tmp]

        context = ""
        for tmp in gold_inds:
            context += gold_inds[tmp]

        acc_len = len(context.split(" "))

        for tmp in sorted_dict:
            if len(all_table_in) + len(all_text_in) >= topn:
                break
            this_sent_ind = int(tmp["ind"].split("_")[1])

            if "table" in tmp["ind"]:
                this_sent = table_row_to_text(table[0], table[this_sent_ind])
            else:
                this_sent = all_text[this_sent_ind]

            if acc_len + len(this_sent.split(" ")) < max_len:
                if "table" in tmp["ind"]:
                    all_table_in[tmp["ind"]] = this_sent
                else:
                    all_text_in[tmp["ind"]] = this_sent

                acc_len += len(this_sent.split(" "))
            else:
                break

        this_model_input = []

        # sorted_dict = sorted(all_table_in.items(), key=lambda kv: int(kv[0].split("_")[1]))
        # this_model_input.extend(sorted_dict)

        # sorted_dict = sorted(all_text_in.items(), key=lambda kv: int(kv[0].split("_")[1]))
        # this_model_input.extend(sorted_dict)

        # original_order
        sorted_dict_table = sorted(all_table_in.items(), key=lambda kv: int(kv[0].split("_")[1]))
        sorted_dict_text = sorted(all_text_in.items(), key=lambda kv: int(kv[0].split("_")[1]))

        for tmp in sorted_dict_text:
            if int(tmp[0].split("_")[1]) < len(pre_text):
                this_model_input.append(tmp)

        for tmp in sorted_dict_table:
            this_model_input.append(tmp)

        for tmp in sorted_dict_text:
            if int(tmp[0].split("_")[1]) >= len(pre_text):
                this_model_input.append(tmp)


        each_data["qa"]["model_input"] = this_model_input
        